In [41]:
import pandas as pd
import numpy as np 
import string

#checks if a given word is of english language
def isEnglishLetter(s):
    try:
        s.encode('ascii')
        return 1
    except:
        return 0

#reads input csv file    
data = pd.read_csv("philippine_file.csv")

In [42]:
##preprocessing

#remove all rows with field of study and industry are undefined
#since those 2 information are the descriptive information for the study
data = data[(data['educations-field_of_study'] != "--undefined--") & (data['industry'] != "--undefined--")]

#gets the significant columns only
columns = ["industry", "educations-degree", "educations-field_of_study", "educations-school-name", "positions-is-current", "num-connections"]
new_data = data[columns]

#changes all work status from undefined to False (assumption: if undefined then the person is not working)
new_data['positions-is-current'] = new_data['positions-is-current'].replace('--undefined--', 'FALSE')

#replace all undefined values to nan
new_data = new_data.replace("--undefined--",np.NaN)
new_data = new_data.reset_index()

#removes rows that are not english
is_englishLetter = []
for i in range(0, new_data.shape[0]):
    is_englishLetter.append(isEnglishLetter(new_data["industry"][i]))
new_data["is_english_letter"] = is_englishLetter
new_data = new_data[(new_data["is_english_letter"] == 1)]

#drop none english letters
del new_data["is_english_letter"]
new_data = new_data.reset_index()
del new_data["level_0"]
del new_data["index"]

#changes to UpperCase all values
for i in columns:
    new_data[i] = [str(j).upper() for j in new_data[i].tolist()]

C:\Users\mychll31\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [44]:
# import enchant
import re
#print re.findall(r"[\w']+", DATA)

# d = enchant.Dict("en_US")
# industry = new_data["industry"]#.unique()

# english_word = []
# for i in range(0, len(industry)):
#     value = industry[i]
#     res = re.findall(r"[\w']+", value)
#     english_word.append(d.check(res[0]))
# new_data["english_word"] = english_word
# new_data = new_data[(new_data["english_word"] == True)]

#drop none english words
# del new_data["english_word"]
# new_data = new_data.reset_index()
# del new_data["index"]
new_data

,industry,educations-degree,educations-field_of_study,educations-school-name,positions-is-current,num-connections
0,EDUCATION MANAGEMENT,BACHELOR OF SCIENCE IN ELEMENTARY EDUCATION,GENERAL,NEW NORTHERN MINDANAO COLLEGES,FALSE,0
1,EDUCATION MANAGEMENT,BACHELOR OF SCIENCE,HOTEL AND RESTAURANT MANAGEMENT,UNIVERSITY OF SANTO TOMAS,TRUE,4
2,MECHANICAL OR INDUSTRIAL ENGINEERING,BSME,MECHANICAL/INDUSTRY,CENTRAL PHILIPPINE UNIVERSITY,TRUE,3
3,COMPUTER HARDWARE,BSME,ENGINEERING,UNIVERSITY OF THE PHILIPPINES,FALSE,36
4,BANKING,BACHELOR,BANKING AND FINANCE,POLYTECHNIC UNIVERSITY OF THE PHILIPPINES,TRUE,0
5,PRIMARY/SECONDARY EDUCATION,NAN,NAN,UNIVERSITY OF THE PHILIPPINES,FALSE,5
6,MARKETING AND ADVERTISING,NAN,BATTERIES,NAN,MESSAGEFORE,0
7,HIGHER EDUCATION,MASTERAL,ENGLISH,DE LA SALLE UNIVERSITY,TRUE,6
8,INFORMATION TECHNOLOGY AND SERVICES,BS,COMPUTER SCIENCE,UP,FALSE,0
9,INFORMATION TECHNOLOGY AND SERVICES,BACHELOR'S DEGREE,BACHELOR OF SCIENCE IN INFORMATION TECHNOLOGY,DAVAO DEL NORTE STATE COLLEGE,FALSE,8


In [70]:
specializations = [{'id': 0, 'label': 'ACCOUNTING/FINANCE', 'word_contains': {'INSURANCE','FINANC', 'ACCOUNTING', 'BANKING', 'PHARMACEUTICALS'}},
                   {'id': 1, 'label': 'ADMIN/HUMAN RESOURCE', 'word_contains': {'ADMIN', 'HUMAN RESOURCE'}},
                   {'id': 2, 'label': 'ARTS/MEDIA', 'word_contains': {'MEDIA', 'ARTS', 'DESIGN'}},
                   {'id': 3, 'label': 'BUILDING/CONSTRUCTION', 'word_contains': {'BUILDING', 'CONSTRUCTION','CONCRETE','MINING'}},
                   {'id': 4, 'label': 'COMPUTER/INFORMATION TECHNOLOGY', 'word_contains': {'TELECOMMUNICATIONS','COMPUTER', 'INFORMATION','INTERNET'}},
                   {'id': 5, 'label': 'EDUCATION/TRAINING', 'word_contains': {'EDUCATION', 'TRAINING'}},
                   {'id': 6, 'label': 'ENGINEERING', 'word_contains': {'ENGINEERING','ARCHITECTURE'}},
                   {'id': 7, 'label': 'HEALTH', 'word_contains': {'MEDICINE', 'HEALTH','MEDICAL'}},
                   {'id': 8, 'label': 'HOTEL/RESTAURANT', 'word_contains': {'FOOD','HOSPITALITY','HOTEL', 'RESTAURANT'}},
                   {'id': 9, 'label': 'MANUFACTURING', 'word_contains': {'MANUFACTURING', 'PRODUCTION','PRINTING'}},
                   {'id': 10, 'label': 'SALES/MARKETING', 'word_contains': {'MARKETING', 'SALES'}},
                   {'id': 11, 'label': 'SCIENCES', 'word_contains': {'LIFE', 'SCIENCES', 'RESEARCH'}},
                   {'id': 12, 'label': 'JOURNALISM', 'word_contains': {'WRITING', 'JOURNALISM'}},
                   {'id': 13, 'label': 'ENTERTAINMENT', 'word_contains': {'ENTERTAINMENT'}},
                   {'id': 14, 'label': 'PUBLIC RELATIONS', 'word_contains': {'LEGAL','RELATIONS', 'LAW'}},
                   {'id': 15, 'label': 'BUSINESS', 'word_contains': {'TRADE', 'REAL'}},
                   {'id': 16, 'label': 'OTHERS', 'word_contains': {''}}]

spe = []
for i in new_data["industry"]:
    val_spe = [l["id"] for l in specializations if any(w in i for w in l["word_contains"])] 
    spe.append(val_spe[0])
    
new_data["specialization"] = spe

#changes all work status from undefined to False (assumption: if undefined then the person is not working)
new_data['positions-is-current'] = new_data['positions-is-current'].replace('FALSE', 0)
new_data['positions-is-current'] = new_data['positions-is-current'].replace('TRUE', 1)

new_data

,industry,educations-degree,educations-field_of_study,educations-school-name,positions-is-current,num-connections,specialization
0,EDUCATION MANAGEMENT,BACHELOR OF SCIENCE IN ELEMENTARY EDUCATION,GENERAL,NEW NORTHERN MINDANAO COLLEGES,0,0,5
1,EDUCATION MANAGEMENT,BACHELOR OF SCIENCE,HOTEL AND RESTAURANT MANAGEMENT,UNIVERSITY OF SANTO TOMAS,1,4,5
2,MECHANICAL OR INDUSTRIAL ENGINEERING,BSME,MECHANICAL/INDUSTRY,CENTRAL PHILIPPINE UNIVERSITY,1,3,6
3,COMPUTER HARDWARE,BSME,ENGINEERING,UNIVERSITY OF THE PHILIPPINES,0,36,4
4,BANKING,BACHELOR,BANKING AND FINANCE,POLYTECHNIC UNIVERSITY OF THE PHILIPPINES,1,0,0
5,PRIMARY/SECONDARY EDUCATION,NAN,NAN,UNIVERSITY OF THE PHILIPPINES,0,5,5
6,MARKETING AND ADVERTISING,NAN,BATTERIES,NAN,MESSAGEFORE,0,10
7,HIGHER EDUCATION,MASTERAL,ENGLISH,DE LA SALLE UNIVERSITY,1,6,5
8,INFORMATION TECHNOLOGY AND SERVICES,BS,COMPUTER SCIENCE,UP,0,0,4
9,INFORMATION TECHNOLOGY AND SERVICES,BACHELOR'S DEGREE,BACHELOR OF SCIENCE IN INFORMATION TECHNOLOGY,DAVAO DEL NORTE STATE COLLEGE,0,8,4
